# Lecture Notes
## What is the difference between loss and metric
- Loss and metric are closely related but metric is the thing you care about. THe loss is what your computer is using as the performance measure to decide how to update model parameters.

## What is overfitting?
- When a model learns the training data too closely, including noise and random flucations, resulting in excellent performance on training data, but poor performance on new, unseen data.
- Caused by overly complex models, forcing memorizatio of training examples.
- Can overcome by splitting off a third validation set (test set), not used for training or metrics. Use after the whole model is complete to verify performance.

## Does using a validation set guarantee we don't overfit?
- No. Every change/run on our data

## What is "transfer learning" and why?
- Transfer learning in machine learning is a technique where knowledge gained from solving one problem is reused to improve performance on a different but related problem.
- When you use transfer learning, you can take advantage of prelearned feature.

## What is p-value?
- In machine learning, a p-value is a statistical measure used to assess the significance of an observed effect or relationship within data.
- One way to go about calculating the p-value would be by simulation. Another would be to use a mathematical formula to jump straight to this number.
- Don't trust p-value ([source](https://www.amstat.org/asa/files/pdfs/p-valuestatement.pdf))

## What do we need to call on a pathlib.Path to delete the file?
- path.unlink()


In [1]:
from fastai.vision.all import *

In [31]:
# Download and decompress pictures of dogs and cats
path = untar_data(URLs.PETS)/'images'

In [32]:
# label our iages as dogs or cats
def is_cat(x): return x[0].isupper()

In [6]:
# Create dataloader
dls = ImageDataLoaders.from_name_func('.', 
    get_image_files(path),
    valid_pct=0.2,
    seed=42,
    label_func=is_cat,
    item_tfms=Resize(192)
)

In [33]:
# Train the model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

epoch,train_loss,valid_loss,error_rate,time


KeyboardInterrupt: 

In [29]:
learn.export('model.pkl')

In [38]:
from fastai.learner import load_learner
from fastai.vision.all import PILImage
import requests, io

learn = load_learner('model.pkl', cpu=True)  # works on CPU/GPU

# from file
pred, idx, probs = learn.predict(PILImage.create('photos/portrait.jpg'))

# from URL
url = "https://as2.ftcdn.net/v2/jpg/00/77/51/81/1000_F_77518136_F88I0v3R2mZsKEgxxXMc4iqXlOjK8OLE.jpg"
img = PILImage.create(io.BytesIO(requests.get(url, timeout=15).content))
pred, idx, probs = learn.predict(img)
print(pred, float(probs[idx]))

True 1.0
